In [ ]:
import uuid


class Example():
    def __init__(self, question, answer, intent=None) -> None:
        self.question = question
        self.answer = answer
        if intent is None:
            self.intent = str(uuid.uuid4())
        else:
            self.intent = intent

In [111]:
examples_path = 'chatbot_epredmet_examples_20072023.txt'

examples = []
k = 0
with open(examples_path, 'r') as f:
    c_intro, c_question, c_answer = None, None, None
    for i, line in enumerate(f.readlines()):
        line = line.strip()
        if line.startswith('!!'):
            c_intro = f'{line[2:]} - '
        elif k % 2 == 0:
            c_question = line
            k += 1
        else:
            c_answer = line
            example = Example(c_intro + c_question, c_answer)
            #example = Example(c_question, c_answer)
            examples.append(example)
            k += 1

In [112]:
from collections import OrderedDict
import yaml


# generate domain file
#with open('domain.yml')
domain_dict = {}
domain_dict['version'] = '3.1'
domain_dict['intents'] = []
domain_dict['responses'] = {}
for example in examples:
    domain_dict['intents'].append(example.intent)
    domain_dict['responses'][f'utter_{example.intent}'] = [{'text': f'"{example.answer}"'}]

In [113]:
with open('domain.yml', 'w') as f:
    yaml.dump(domain_dict, f, allow_unicode=True, width=float("inf"))

In [114]:
from yaml.resolver import BaseResolver

class AsLiteral(str):
  pass

def represent_literal(dumper, data):
  return dumper.represent_scalar(BaseResolver.DEFAULT_SCALAR_TAG,
      data, style="|")

yaml.add_representer(AsLiteral, represent_literal)

In [115]:
nlu_dict = {}
nlu_dict['version'] = "3.1"
nlu_dict['nlu'] = []
for example in examples:
    nlu_example = {}
    nlu_example['intent'] = example.intent
    example_str = [f'{example.question}']
    example_str = AsLiteral(yaml.dump(example_str, allow_unicode=True, width=float("inf")))
    nlu_example['examples'] = example_str
    nlu_dict['nlu'].append(nlu_example)

In [116]:
with open('data/nlu.yml', 'w') as f:
    yaml.dump(nlu_dict, f, allow_unicode=True, width=float("inf"))

In [117]:
stories_dict = {}
stories_dict['version'] = "3.1"
stories_dict['stories'] = []
for i, example in enumerate(examples):
    story = {'story': str(i)}
    story['steps'] = [{'intent': example.intent}, {'action': f'utter_{example.intent}'}]
    stories_dict['stories'].append(story)

In [118]:
with open('data/stories.yml', 'w') as f:
    yaml.dump(stories_dict, f, allow_unicode=True, width=float("inf"))

In [119]:
rules_dict = {}
rules_dict['version'] = "3.1"
rules_dict['rules'] = []
for i, example in enumerate(examples):
    rule = {'rule': str(i)}
    rule['steps'] = [{'intent': example.intent}, {'action': f'utter_{example.intent}'}]
    rules_dict['rules'].append(rule)

In [120]:
with open('data/rules.yml', 'w') as f:
    yaml.dump(rules_dict, f, allow_unicode=True, width=float("inf"))